In [63]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier

In [64]:
df = pd.read_csv('titanic.csv')

In [65]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [66]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [67]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']),
                                                   df['Survived'],
                                                   test_size=0.2,
                                                   random_state=42)

In [68]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [69]:
y_train.head()

336    0
31     0
84     0
287    0
317    0
Name: Survived, dtype: int64

In [70]:
df.isnull().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [71]:
si_age = SimpleImputer()
si_fare = SimpleImputer(strategy='most_frequent')
si_embarked = SimpleImputer(strategy='most_frequent')

x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_fare = si_fare.fit_transform(x_train[['Fare']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_fare = si_fare.fit_transform(x_test[['Fare']])
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])

In [72]:
# x_train_

In [73]:
ohe_sex = OneHotEncoder(handle_unknown='ignore')
ohe_embarked = OneHotEncoder(handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']]).toarray()
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked).toarray()

x_test_sex = ohe_sex.fit_transform(x_test[['Sex']]).toarray()
x_test_embarked = ohe_embarked.fit_transform(x_test_embarked).toarray()

In [74]:
x_test_embarked.shape

(84, 3)

In [75]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0,S
31,2,male,24.0,2,0,31.5,S


In [76]:
x_train_rem = x_train.drop(columns=['Sex','Age','Embarked','Fare'])
x_test_rem = x_test.drop(columns=['Sex','Age','Embarked','Fare'])

In [89]:
x_test_rem.head(2)

,Pclass,SibSp,Parch
321,3,0,0
324,1,0,0


In [78]:
x_train_transform = np.concatenate((x_train_rem,x_train_fare,x_train_age,x_train_sex,x_train_embarked),axis=1)

In [88]:
# x_train_transform

array([[2., 0., 0., ..., 0., 0., 1.],
       [2., 2., 0., ..., 0., 0., 1.],
       [2., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 1., 0., 0.],
       [2., 0., 0., ..., 0., 0., 1.],
       [3., 0., 0., ..., 0., 1., 0.]])

In [79]:
x_test_transform = np.concatenate((x_test_rem,x_test_fare,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [80]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transform, y_train)

DecisionTreeClassifier()

In [81]:
y_pred = clf.predict(x_test_transform)

In [82]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

1.0

### We csn use PICKLE to export model for further use in other cs field like web dev, app dev, etc

In [83]:
import pickle

In [86]:
pickle.dump(ohe_sex,open('models/ohe_sex.pkl','wb'))
pickle.dump(ohe_sex,open('models/ohe_embarked.pkl','wb'))
pickle.dump(clf,open('models/clf.pkl','wb'))